<a href="https://colab.research.google.com/github/LiadTssf/cloud-project/blob/main/hw2/cloud_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.1 MB/s eta 0:00:00


In [2]:
from IPython.display import display, clear_output, FileLink
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from firebase import firebase

from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
from reportlab.lib.units import inch
from google.colab import files


# Functions

## Database

In [3]:
# Get data from firebase
def get_data_from_firebase(url, database, data):
    FBconn = firebase.FirebaseApplication(url, None)
    data = FBconn.get(database, data)
    return data

In [4]:
# Function to get set array of onshape documents
def get_documents_set(data):
    documents = set()
    for item in data:
        document = item.get("Document")
        if document is not None:
            documents.add(document)
    return sorted(documents)

In [5]:
# Function to get users associated with selected projects
def get_users_by_projects(data, selected_projects):
    users = set()
    for item in data:
        if item.get("Document") in selected_projects:
            users.add(item.get("User"))
    return users

## Buttons

In [6]:
# Function to update team member tab content and navigate to it
def update_members(button):
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    users = get_users_by_projects(data, selected_projects)
    user_checkboxes.clear()
    user_checkboxes.extend([
        widgets.Checkbox(
            value=False,
            description=str(user),
        ) for user in users])

    members_tab.children = [members_header] + user_checkboxes + [widgets.HBox([select_members_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    tabs.selected_index = 1  # Navigate to the "Select Team Member" tab

In [7]:
def update_analytics(button):
    analytics_content.children = (
        [graph_type_dropdown,
         plots_output,
         widgets.HBox([generate_button, export_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    )

    with plots_output:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.pie([35, 25, 20, 20], labels=['A', 'B', 'C', 'D'], autopct='%1.1f%%')
        ax.set_title("No Graph Selected")
        plt.show()

    tabs.selected_index = 2  # Navigate to the "Analytics" tab

In [8]:
def generate_graph(button):
    global generated_figures
    generated_figures.clear()  # Clear previous figures
    # clear outputs
    with plots_output:
        clear_output(wait=True)

    # Get current selections from UI elements
    selected_users = [checkbox.description for checkbox in user_checkboxes if checkbox.value]
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    graph_type = graph_type_dropdown.value

    # Validate selections and perform graph generation
    if selected_users and selected_projects:

        #**************** Contribution Breakdown ******************
        if graph_type == 'Contribution Breakdown':
            contribution_breakdown(selected_users, selected_projects)

        #**************** Activity Timeline ******************
        elif graph_type == 'Activity Timeline':
            activity_timeline(selected_users, selected_projects)

        #**************** Task Distribution ******************
        elif graph_type == 'Task Distribution':
            task_distribution(selected_users, selected_projects)

        #**************** user activity by tab ******************
        elif graph_type == 'user activity by tab':
            user_activity_by_tab(selected_users, selected_projects)
        #****************  user_productivity_heatmap ******************
        elif graph_type == 'user productivity heatmap':
            user_productivity_heatmap(selected_users, selected_projects)
        #*********************** All Graphs *********************
        elif graph_type == 'All':
            all_graphs(selected_users, selected_projects)

        else:
            print("Invalid Graph Type selected.")

    else:
        print("Please select at least one user and one project.")





generated_figures = []

## Graphs

In [9]:
def contribution_breakdown(selected_users, selected_projects):
    global generated_figures
    activity = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity.setdefault(user, [0]*12)[month - 1] += 1

    labels = list(activity.keys())
    sizes = np.sum(list(activity.values()), axis=1)

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    ax.set_title("Contribution Breakdown")
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [10]:
def activity_timeline(selected_users, selected_projects):
    global generated_figures
    activity = {user: [0]*12 for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity[user][month - 1] += 1

    fig, ax = plt.subplots()
    for user, activity_data in activity.items():
        ax.plot(activity_data, label=user)
    ax.set_xticks(range(12))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.set_title("Activity Timeline")
    ax.legend()
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [11]:
def create_task_dataframe(selected_users, selected_projects):
    tasks = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            task_type = item.get("Description").split(' ')[0]  # Example extraction
            tasks.setdefault(user, {}).setdefault(task_type, 0)
            tasks[user][task_type] += 1

    # Convert tasks dictionary to a DataFrame
    df = pd.DataFrame(tasks).fillna(0).astype(int)

    # Calculate the sum of all actions and add it as a new row
    total_actions = df.sum(axis=0)
    df.loc['Total Actions'] = total_actions

    # Rename the index to "Actions"
    df.index.name = 'Actions'

    return df


In [12]:
def task_distribution(selected_users, selected_projects):
    global generated_figures
    df = create_task_dataframe(selected_users, selected_projects)

    # Create a figure from the DataFrame
    fig, ax = plt.subplots(figsize=(12, 6))
    df.plot(kind='bar', ax=ax)
    ax.set_title("Task Distribution")
    ax.legend(title="Users")
    plt.tight_layout()
    generated_figures.append(fig)

    with plots_output:
        display(df)
        plt.show()


In [13]:
def user_activity_by_tab(selected_users, selected_projects):
    global generated_figures
    activity = {user: {} for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        tab = item.get("Tab")
        if user in selected_users and project in selected_projects:
            activity[user][tab] = activity[user].get(tab, 0) + 1

    fig, ax = plt.subplots(figsize=(12, 6))
    bottom = np.zeros(len(selected_users))
    for tab in set(tab for user_data in activity.values() for tab in user_data):
        values = [activity[user].get(tab, 0) for user in selected_users]
        ax.bar(selected_users, values, label=tab, bottom=bottom)
        bottom += values

    ax.set_xlabel('Users')
    ax.set_ylabel('Number of Actions')
    ax.set_title("User Activity by Tab")
    ax.legend(title="Tabs")
    plt.xticks(rotation=45, ha='right')
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [14]:
import seaborn as sns

def user_productivity_heatmap(selected_users, selected_projects):
    global generated_figures
    activity = {user: [0]*24 for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            hour = int(item.get("Time").split()[1].split(':')[0])
            activity[user][hour] += 1

    df = pd.DataFrame(activity).T

    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(df, cmap="YlOrRd", ax=ax)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Users')
    ax.set_title("User Productivity Heatmap")
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [15]:
def all_graphs(selected_users, selected_projects):
    contribution_breakdown(selected_users, selected_projects)
    activity_timeline(selected_users, selected_projects)
    task_distribution(selected_users, selected_projects)
    user_activity_by_tab(selected_users, selected_projects)
    user_productivity_heatmap(selected_users, selected_projects)

In [16]:
from reportlab.platypus import SimpleDocTemplate, Image, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

def download_graphs(button):
    global generated_figures

    if not generated_figures:
        print("No graphs were generated. Please generate graphs before attempting to download.")
        return

    # Create a BytesIO object to store the PDF
    buffer = BytesIO()

    # Create the PDF document
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    elements = []

    # Iterate through all the generated figures
    for fig in generated_figures:
        # Save the figure to a BytesIO object
        img_buffer = BytesIO()
        fig.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight')
        img_buffer.seek(0)

        # Create an Image object and add it to the elements list
        img = Image(img_buffer)
        img.drawHeight = 6*inch
        img.drawWidth = 8*inch
        elements.append(img)

    # Build the PDF
    doc.build(elements)

    # Move the buffer's cursor to the beginning
    buffer.seek(0)

    # Generate a filename
    filename = f"onshape_analytics_{len(generated_figures)}_graphs.pdf"

    # Write the PDF content to a file
    with open(filename, 'wb') as f:
        f.write(buffer.getvalue())

    # Use google.colab.files to initiate the download
    files.download(filename)

    print(f"PDF file '{filename}' has been created and downloaded.")

    # Clear the generated figures list
    generated_figures.clear()

# Program

## Initialisation

In [17]:
data = get_data_from_firebase('https://test100000-ccdb4-default-rtdb.asia-southeast1.firebasedatabase.app/', '/team_data2', 'data')
data_documents = get_documents_set(data)

## Widgets

In [18]:
style = widgets.HTML("""
    <style>
      html, body {
        height: 100vh;
        margin: 0; /* Remove default margin */
        padding: 0; /* Remove default padding */
      }
      body {
        font-family: 'Times New Roman', serif;
        background: rgb(8,129,113);
        background: linear-gradient(90deg, rgba(8,129,113,1) 0%, rgba(237,237,237,1) 67%);
        border: none; /* Remove any border */
      }
      ul {
        padding-left: 20px;
        margin-top: 10px;
        margin-bottom: 10px;
        list-style-type: disc;
        justify-content: space-between;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
        flex: 1 1 var(--jp-widgets-horizontal-tab-width);
        font-weight: bold;
        background-color: transparent;
        color: black;
        border: 1px solid black;
        border-radius: 5px;
        padding: 10px;
        transition: background-color 0.3s, border 0.3s;
        hover:
      }
        .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab:hover {
         background-color: black; /* Black background on hover */
          color: white; /* White text on hover */
          border: 1px solid black; /* Black border on hover */
          cursor: pointer; /* Make it clickable */
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current {
        background-color: black; /* Black background for active tab */
        color: white; /* White text for active tab */
      }
      li {
        font-size: 20px !important;
        font-weight: bold;
        margin-bottom: 5px;
        padding: 10px;
        gap: 10px;
        background-color: #ffffff; /* White background for list items */
        border: solid 1px #dcdcdc; /* Light gray border for list items */
        border-radius: 5px; /* Rounded corners for list items */
      }
      h3 {
        text-decoration: underline;
      }
      h1 {
        text-decoration: underline;
        padding: 20px;
        color: black;
      }
      span {
        font-size: 20px !important;
      }
      select, option {
        font-family: 'Times New Roman', serif;
        font-size: 20px !important;

      }
      .widget-label {
        font-size: 22px;
      }
      .widget-button {
        background-color: #007bff;
        color: white;
        height: 50px;
        font-size: 24px;
        font-weight: bold;
        border: none;
        border-radius: 5px;
        padding: 10px;
        cursor: pointer;
      }
      .widget-button:hover {
        background-color: #0056b3;
      }
      .lm-Widget.p-Widget.lm-Panel.p-Panel.p-TabPanel-tabContents.widget-tab-contents {
        background: rgb(8,129,113);
        background: linear-gradient(90deg, rgba(8,129,113,1) 0%, rgba(237,237,237,1) 67%);
        flex-grow: 1;
        border: none;
      }
    </style>
""")


In [19]:
## projects tab

# header
project_header = widgets.HTML("<h2>Select Projects</h2>")

# checkboxs
project_checkboxes = [
    widgets.Checkbox(
        value=False,
        description=str(doc),
    ) for doc in data_documents if doc is not None
]

# select button
select_projects_button = widgets.Button(
    description="Next",
)

select_projects_button.on_click(update_members)

# layout
projects_tab = widgets.VBox([
        project_header,
        widgets.VBox(project_checkboxes),
        widgets.HBox([select_projects_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))
    ])

In [20]:
## members tab

user_checkboxes = []

# start empty layout
members_tab = widgets.VBox([widgets.Label("Select projects to continue")])

# header
members_header = widgets.HTML("<h2>Select Team Members</h2>")

# select button
select_members_button = widgets.Button(
    description="Next",
)
select_members_button.on_click(update_analytics)

# full layout in function update_members()

In [21]:
## analytics tab

plots_output = widgets.Output()

# header
members_header = widgets.HTML("<h2>Analytics</h2>")

# Dropdown menu
graph_type_dropdown = widgets.Dropdown(
    options=['Contribution Breakdown', 'Activity Timeline', 'Task Distribution', 'user activity by tab', 'user productivity heatmap', 'All'],
    description='Select Graph Type:',
    style={'description_width': 'initial', 'description_font_size': '40px'},
    layout={'width': '450px'}
)



# generate button
generate_button = widgets.Button(
    description="Generate",
)
# generate button
export_button = widgets.Button(
    description="Export",
)

# Spacer
spacer = widgets.Box(layout=widgets.Layout(width='20px'))
export_button.on_click(download_graphs)
generate_button.on_click(generate_graph)

# start empty layout
analytics_content = widgets.VBox([widgets.Label("Select projects and members to continue")])

# full layout in function update_analytics()

## Main

In [22]:
# header
header = widgets.HTML("<h1>OnShape Analytics Program</h1>")

# tabs
tabs = widgets.Tab()
tabs.children = [
    projects_tab,
    members_tab,
    analytics_content
]
tabs.set_title(0, 'Select Projects')
tabs.set_title(1, 'Select Members')
tabs.set_title(2, 'Analytics')

# layout and display
app_layout = widgets.VBox([style, header, tabs])

display(app_layout)